# Plot example data for a single patient

In [ ]:
# Import libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import psycopg2
%matplotlib inline

In [ ]:
# Config
sqluser = 'alistairewj'
dbname = 'mimic'
schema_name = 'public,mimiciii'

# append the search path to the beginning of each query
query_schema = 'set search_path to ' + schema_name + ';'

In [ ]:
# Connect to MIMIC
con = psycopg2.connect(dbname=dbname, user=sqluser)

In [ ]:
# Load angus
query = query_schema + """
-- get first icu stay for adult patients
with t1 as 
(
select ie.icustay_id, ie.hadm_id, ie.intime, ie.outtime
, ROW_NUMBER() over (partition by ie.subject_id order by ie.intime) as icustay_num
from icustays ie
inner join admissions adm
on ie.hadm_id = adm.hadm_id
-- adult patient
inner join patients pat
on ie.subject_id = pat.subject_id
and ie.intime > pat.dob + interval '1' year
)
SELECT t1.hadm_id
, si.charttime as first_antibiotic_time
, si.positiveculture as positive_culture
, ms.sepsis as martin
, ang.angus
FROM t1
left join micro_first si
on t1.hadm_id = si.hadm_id
and si.charttime between t1.intime - interval '1' day and t1.intime + interval '1' day
left join angus_sepsis ang
on t1.hadm_id = ang.hadm_id
left join martin_sepsis ms
on t1.hadm_id = ms.hadm_id
where t1.icustay_num = 1
"""
df = pd.read_sql_query(query,con)

In [ ]:
print('{:5g} first ICU stay for adults.'.format(
        df['hadm_id'].count()))
print('{:5g} with suspected sepsis (blood culture within 24 hours of admission)'.format(
        df['first_antibiotic_time'].count()))
print('{:5g} with a positive blood culture'.format(
    df['positive_culture'].sum()))
print('{:5g} with Angus'.format(
    df['angus'].sum()))
print('{:5g} with Angus and suspected sepsis'.format(
    ((df['angus']==1)&(~df['first_antibiotic_time'].isnull())).sum()))
print('{:5g} with Martin'.format(
    df['martin'].sum()))
print('{:5g} with Martin and suspected sepsis'.format(
    ((df['martin']==1)&(~df['first_antibiotic_time'].isnull())).sum()))

In [ ]:
# venn diagram with matplotlib-venn
from matplotlib_venn import venn3

idx = df['martin']==1
set1 = set(df.loc[idx,'hadm_id'])
idx = df['angus']==1
set2 = set(df.loc[idx,'hadm_id'])
idx = ~df['first_antibiotic_time'].isnull()
set3 = set(df.loc[idx,'hadm_id'])

plt.figure(figsize=[12,12])
plt.rcParams.update({'font.size': 14})
venn3([set1, set2, set3], ('Martin criteria', 'Angus criteria', 'Suspected of infection'))
plt.show()

In [ ]:
con.close()